## Codercise S.1.1 - Modular equivalence

In [3]:
def is_equivalent(a, b, m):
    """Return a boolean indicating whether the equivalence is satisfied.

    Args:
        a (int): First number to check the equivalence.
        b (int): Second number to check the equivalence.
        m (int): Modulus of the equivalence.

    Returns:
        bool: True if a = b (m), False otherwise.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    if (a-b)%m == 0:
        return True
    else: return False


print(f"13 = 8 (3) is {is_equivalent(13, 8, 3)}")
print(f"13 = 7 (6) is {is_equivalent(13, 7, 6)}")

13 = 8 (3) is False
13 = 7 (6) is True


## Codercise S.1.2 - The inverse in modular arithmetic

In [4]:

def has_inverse(a, m):
    """Returns a boolean indicating whether a number has an inverse modulo m.

    Args:
        a (int): Number to find the inverse modulus m.
        m (int): Modulus of the equivalence.

    Returns:
        bool: True if c exists (ac = 1 (m)), False otherwise
    """

    ##################
    # YOUR CODE HERE #
    ##################
    def computeGCD(x, y):
        while(y):
           x, y = y, x % y
        return abs(x)
    
    if computeGCD(a,m) == 1:
        return True
    else:
        return False
    

print("(5,15)", has_inverse(5, 15))
print("(7,15)", has_inverse(7, 15))


(5,15) False
(7,15) True


## Codercise S.2.1 - Nontrivial square roots

In [5]:
def nontrivial_square_root(m):
    """Return the first nontrivial square root modulo m.

    Args:
        m (int): number we want to find the root square.

    Returns:
        (int): the first non trivial root square of m
    """

    ##################
    # YOUR CODE HERE #
    ##################

    for i in range(2, m - 2):
        if i**2 % m == 1:
            return i


print(nontrivial_square_root(391))

137


## Codercise S.2.2 - Prime factors

In [6]:
def factorization(N):
    """Return the factors of N.

    Args:
        N (int): number we want to factor.

    Returns:
        array[int]: [p,q] factors of N.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    def computeGCD(x, y):
        while(y):
           x, y = y, x % y
        return abs(x)

    root = nontrivial_square_root(N)
    return computeGCD(root+1,N), computeGCD(root-1,N)

N = 391
p, q = factorization(N)
print(f"{N} = {p} x {q}")


391 = 23 x 17


## Codercise S.3.1 - The QPE component

In [10]:
import pennylane as qml

def U():
    qml.SWAP(wires=[2, 3])
    qml.SWAP(wires=[1, 2])
    qml.SWAP(wires=[0, 1])
    for i in range(4):
        qml.PauliX(wires=i)


matrix = qml.matrix(U, wire_order=range(4))()

n_target_wires = 4
target_wires = range(n_target_wires)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)


dev = qml.device("default.qubit", shots=1, wires=n_target_wires + n_estimation_wires)


@qml.qnode(dev)
def circuit(matrix):
    """Return a sample after taking a shot at the estimation wires.

    Args:
        matrix array[array[(complex)]]: matrix representation of U.

    Returns:
        array[(float)]: a sample after taking a shot at the estimation wires.

    """

    ##################
    # YOUR CODE HERE #
    ##################

    # CREATE THE INITIAL STATE |0001>

    qml.PauliX(wires=3)

    # USE THE SUBROUTINE QUANTUM PHASE ESTIMATION

    qml.QuantumPhaseEstimation(matrix, target_wires=target_wires, estimation_wires=estimation_wires)

    ##################

    return qml.sample(wires=estimation_wires)


def get_phase(matrix):
    binary = "".join([str(b) for b in circuit(matrix)])
    return int(binary, 2) / 2**n_estimation_wires


for i in range(5):
    print(circuit(matrix))
    print(f"shot {i+1}, phase:", get_phase(matrix))

[1 1 0]
shot 1, phase: 0.0
[1 1 0]
shot 2, phase: 0.5
[0 1 0]
shot 3, phase: 0.75
[0 0 0]
shot 4, phase: 0.5
[0 0 0]
shot 5, phase: 0.0


## Codercise S.3.2 - Determining the period

In [11]:
import fractions

def U():
    qml.SWAP(wires=[2, 3])
    qml.SWAP(wires=[1, 2])
    qml.SWAP(wires=[0, 1])
    for i in range(4):
        qml.PauliX(wires=i)


matrix = qml.matrix(U, wire_order=range(4))()

target_wires = range(4)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)


def get_period(matrix):
    """
    Return the period of the previous state using the already defined get_phase function.

    Args:
        matrix array[array(complex)]: matrix associated with the operator U

    Returns:
        (int): Desired period.

    """

    shots = 10
    ##################
    # YOUR CODE HERE #
    ##################

    r = 0
    aux = 2 ** len(estimation_wires)

    for _ in range(shots):
        r = max(r, fractions.Fraction(get_phase(matrix)).limit_denominator(aux).denominator)

    return r


print(get_period(matrix))

4


## Codercise S.3.3 - A sanity check

In [12]:
def U():
    qml.SWAP(wires=[2, 3])
    qml.SWAP(wires=[1, 2])
    qml.SWAP(wires=[0, 1])
    for i in range(4):
        qml.PauliX(wires=i)


dev = qml.device("default.qubit", wires=4)


@qml.qnode(dev)
def circuit():
    """
    Returns:
        array[(float)]: probs of each basic state

    """

    ##################
    # YOUR CODE HERE #
    ##################

    qml.PauliX(wires=3)

    for _ in range(4):
        U()

    return qml.probs(wires=range(4))


print(circuit())

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Codercise S.4.1 - Period finding and square roots

In [13]:
def is_coprime(a, N):
    """Determine if two numbers are coprime.

    Args:
        a (int): First number to check if is coprime with the other.
        N (int): Second number to check if is coprime with the other.

    Returns:
        bool: True if they are coprime numbers, False otherwise.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    def GCD(a, b):
        if(b == 0):
            return abs(a)
        else:
            return GCD(b, a % b)

    return GCD(a, N) == 1
        


def is_odd(r):
    """Determine if a number is odd.

    Args:
        r (int): Integer to check if is an odd number.

    Returns:
        bool: True if it is odd, False otherwise.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    return r % 2 == 1


def is_not_one(x, N):
    """Determine if x is not +- 1 modulo N.

    Args:
        N (int): Modulus of the equivalence.
        x (int): Integer to check if it is different from +-1 modulo N.

    Returns:
        bool: True if it is different, False otherwise.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    return x%N != 1 and x%N != N-1

print("3 and 12 are coprime numbers: ", is_coprime(3, 12))
print("5 is odd: ", is_odd(5))
print("4 is not one mod 5: ", is_not_one(4, 5))

3 and 12 are coprime numbers:  False
5 is odd:  True
4 is not one mod 5:  False


## Codercise S.4.2 - Shor's algorithm

In [ ]:
import numpy as np
def shor(N):
    """
    Return the factorization of a given integer.

    Args:
       N (int): integer we want to factorize.

    Returns:
        array[(int)]: [p,q] Prime factors of N.

    """

    ##################
    # YOUR CODE HERE #
    ##################

    period = 1
    while is_odd(period):
        a = np.random.randint(2, N - 2)

        if not is_coprime(a, N):
            p = np.gcd(a, N)
            return [p, N // p]

        matrix = get_matrix_a_mod_N(a, N)
        period = get_period(matrix, N)

        x = (a ** (period // 2)) % N

        if is_not_one(x, N) and not is_odd(period):
            p = np.gcd(x - 1, N)
            q = np.gcd(x + 1, N)
            return [p, q]

        period = 1


print(shor(21))